In [18]:
# Author: Xianhui
#

from __future__ import print_function
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## multi-class classification with Keras and TensorFlow
https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

In [3]:
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.optimizers import SGD, Adam
from keras.losses import categorical_crossentropy, binary_crossentropy
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

C:\Users\cyret\Anaconda3\lib\site-packages\h5py\__init__.py:72: UserWarning: h5py is running against HDF5 1.10.2 when it was built against 1.10.3, this may cause problems
  '{0}.{1}.{2}'.format(*version.hdf5_built_version_tuple)
Using TensorFlow backend.


In [31]:
_ = load_iris()
X, y = _.data, _.target
print(X.shape, y.shape)
num_classes = len(np.unique(y))
encoder = LabelEncoder()
encoder.fit(y)
encoded_y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)
dummy_y[:5]

(150, 4) (150,)


array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.33, random_state=42)

In [33]:
def base_model(input_dim=4, num_classes=3):
    '''
    set up keras neural network for multi-class
    input_dim: number of features
    num_classes: output layer units, for multi-class > 2
    '''
    model = Sequential()
    model.add(Dense(units=16, input_dim=input_dim))
    model.add(Activation('relu'))
    # model.add(Dense(units=16))
    # model.add(Activation('softmax'))
    # model.add(Dense(units=16))
    # model.add(Activation('sigmoid'))
    model.add(Dense(num_classes, activation='softmax'))  # 3 units for 3 classes
    adam = Adam(lr=0.001)
    model.compile(optimizer=adam, loss=binary_crossentropy, metrics=['accuracy'])
    return model

In [34]:
model = base_model()
model.fit(X_train, y_train, epochs=150, batch_size=32, validation_split=0.1, verbose=1)

Train on 90 samples, validate on 10 samples
Epoch 1/150
90/90 [==============================] - 0s 4ms/step - loss: 0.4700 - acc: 0.8037 - val_loss: 0.4544 - val_acc: 0.8333
Epoch 2/150
90/90 [==============================] - 0s 88us/step - loss: 0.4567 - acc: 0.8407 - val_loss: 0.4399 - val_acc: 0.8667
Epoch 3/150
90/90 [==============================] - 0s 88us/step - loss: 0.4450 - acc: 0.8741 - val_loss: 0.4299 - val_acc: 0.8667
Epoch 4/150
90/90 [==============================] - 0s 132us/step - loss: 0.4418 - acc: 0.8704 - val_loss: 0.4243 - val_acc: 0.8667
Epoch 5/150
90/90 [==============================] - 0s 83us/step - loss: 0.4325 - acc: 0.8630 - val_loss: 0.4184 - val_acc: 0.8667
Epoch 6/150
90/90 [==============================] - 0s 127us/step - loss: 0.4270 - acc: 0.8593 - val_loss: 0.4124 - val_acc: 0.8667
Epoch 7/150
90/90 [==============================] - 0s 94us/step - loss: 0.4204 - acc: 0.8630 - val_loss: 0.4054 - val_acc: 0.8667
Epoch 8/150
90/90 [============

In [38]:
model.predict(X_test)[:5], y_test[:5]

(array([[1.9799819e-02, 7.8865838e-01, 1.9154181e-01],
        [9.7696006e-01, 2.3037234e-02, 2.6397220e-06],
        [9.1145894e-06, 2.3434747e-02, 9.7655606e-01],
        [1.3703571e-02, 7.1773338e-01, 2.6856312e-01],
        [1.2538395e-02, 8.6152124e-01, 1.2594044e-01]], dtype=float32),
 array([[0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.]], dtype=float32))

In [35]:
## accuracy of keras model
np.sum(np.array(model.predict(X_train) > 0.5).astype(float) == y_train) / y_train.shape[0] / y_train.shape[1]

0.9833333333333334

In [36]:
np.sum(np.array(model.predict(X_test) > 0.5).astype(float) == y_test) / y_test.shape[0] / y_test.shape[1]

1.0

In [114]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

In [9]:
from keras.wrappers.scikit_learn import KerasClassifier
estimator = KerasClassifier(build_fn=base_model, epochs=150, batch_size=32, verbose=1)
estimator.fit(X_train, y_train)
# results = cross_val_score(model, X, dummy_y, cv=kfold, scoring=accuracy_score)
# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/150
100/100 [==============================] - 0s 2ms/step - loss: 0.7572 - acc: 0.5400
Epoch 2/150
100/100 [==============================] - 0s 79us/step - loss: 0.7010 - acc: 0.5400
Epoch 3/150
100/100 [==============================] - 0s 74us/step - loss: 0.6572 - acc: 0.6567
Epoch 4/150
100/100 [==============================] - 0s 89us/step - loss: 0.6210 - acc: 0.7700
Epoch 5/150
100/100 [==============================] - 0s 119us/step - loss: 0.5943 - acc: 0.7633
Epoch 6/150
100/100 [==============================] - 0s 69us/step - loss: 0.5726 - acc: 0.7467
Epoch 7/150
100/100 [==============================] - 0s 64us/step - loss: 0.5565 - acc: 0.7233
Epoch 8/150
100/100 [==============================] - 0s 64us/step - loss: 0.5443 - acc: 0.6867
Epoch 9/150
100/100 [==============================] - 0s 124us/step - loss: 0.5333 - acc: 0.6900
Epoch 10/150
100/100 [==============================] - 0s 79us/step - loss: 0.5248 - acc: 0.7267
Epoch 11/150
100/100 [======

In [16]:
accuracy_score(y, estimator.predict(X))

150/150 [==============================] - 0s 36us/step


0.9666666666666667

In [17]:
# print(f'accuracy score {accuracy_score(y_train, estimator.predict(X_train)).round(2)}')
# print(f'accuracy score {accuracy_score(y_test, estimator.predict(X_test)).round(2)}')